# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm

import ast
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# Data

In [ ]:
# Y_RealCase_train = pd.read_csv('Data/Y_train_std.csv', header=None, delimiter=',')
# df_new = Y_RealCase_train.drop(Y_RealCase_train.columns[-18:], axis=1)
# Y_data_train = df_new.drop(df_new.columns[17], axis=1)

In [ ]:
# np.savetxt("Y_RealCase_train.csv", Y_data_train.values, delimiter=",", fmt="%.8f")

In [2]:
X_train = pd.read_csv('RealCase/RealCase_X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('RealCase/RealCase_X_test.csv', header=None, delimiter=',').values

Y_train_pca = pd.read_csv('RealCase/RealCase_Y_train_pca.csv', header=None, delimiter=',').values
Y_test_pca = pd.read_csv('RealCase/RealCase_Y_test_pca.csv', header=None, delimiter=',').values
Realcase_data_pca = pd.read_csv('RealCase/RealCase_Y_pca.csv', header=None, delimiter=',').values

Y_train = pd.read_csv('RealCase/RealCase_Y_train_std.csv', header=None, delimiter=',').values
Y_test = pd.read_csv('RealCase/RealCase_Y_test_std.csv', header=None, delimiter=',').values
Realcase_data = pd.read_csv('RealCase/RealCase_Y.csv', header=None, delimiter=',').values



In [3]:
train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_pca = torch.tensor(Y_train_pca, dtype=torch.float32)
test_y_pca = torch.tensor(Y_test_pca, dtype=torch.float32)
realcase_y_pca = torch.tensor(Realcase_data_pca, dtype=torch.float32)

train_y = torch.tensor(Y_train, dtype=torch.float32)
test_y = torch.tensor(Y_test, dtype=torch.float32)
realcase_y = torch.tensor(Realcase_data, dtype=torch.float32)

torch.set_default_dtype(torch.float32)

# Emulator

In [4]:
# Device = 'cpu'

Device = 'cuda'

In [20]:
Row_peak = Realcase_data_pca[0]

col_min = np.nanmin(Y_train_pca, axis=0)
col_max = np.nanmax(Y_train_pca, axis=0)

maskrow_11 = (Row_peak >= col_min) & (Row_peak <= col_max)

Peak_case = Row_peak[maskrow_11]
Y_train_pca_peak = Y_train_pca[:,maskrow_11]
Y_test_pca_peak = Y_test_pca[:,maskrow_11]

In [21]:
train_y_pca_peak = torch.tensor(Y_train_pca_peak, dtype=torch.float32)
test_y_pca_peak = torch.tensor(Y_test_pca_peak, dtype=torch.float32)
realcase_y_pca_peak = torch.tensor(Peak_case, dtype=torch.float32)

## MVGP

In [ ]:
checkpoint = torch.load('multitask_gp_checkpoint_Realcase_pca.pth', map_location=Device)
model_params = checkpoint['model_params']

MVGP_models = GP_models.MultitaskVariationalGP(train_x, train_y_pca, 
                                               num_latents=model_params['num_latents'],
                                               num_inducing=model_params['num_inducing'],  
                                               covar_type=model_params['covar_type']).to(Device)

MVGP_models.load_state_dict(checkpoint['model_state_dict'])

MVGP_likelihoods = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y_pca.shape[1]).to(Device)
MVGP_likelihoods.load_state_dict(checkpoint['likelihood_state_dict'])

MVGP_models.eval()
MVGP_likelihoods.eval()


In [7]:
MVGP_models, MVGP_likelihoods = Training.train_MultitaskVGP_minibatch(
                train_x=train_x.to(Device),
                train_y=train_y_pca_peak.to(Device),
                covar_type='RBF',
                num_latents=36,
                num_inducing=400,
                lr_hyper=0.01,
                lr_variational=0.1,
                num_iterations=10000,
                patience=10,
                device=Device,
                batch_size=512,
                eval_every=100,
                eval_batch_size=1024
            )

Training: 100%|██████████| 10000/10000 [26:58<00:00,  6.18it/s, full_loss=-19.9]


In [8]:
checkpoint = {
    'model_state_dict': MVGP_models.state_dict(),
    'likelihood_state_dict': MVGP_likelihoods.state_dict()
}

torch.save(checkpoint, 'multitask_gp_checkpoint_Realcase_pca_peak.pth')

# PE

In [11]:
row_idx = 0

input_point = realcase_y_pca_peak

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_GPU(input_point, train_x, train_y_pca_peak, k=100)

bounds = bound.get_bounds(local_train_x)

In [13]:

estimated_params_tmp, Loss_tmp = Estimation.multi_start_estimation(
        MVGP_models, MVGP_likelihoods, row_idx, realcase_y_pca_peak, bounds,
        Estimation.estimate_params_for_one_model_Adam, num_starts=16, num_iterations=2000, lr=0.01,
        patience=15, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
    )

In [14]:
estimated_params_tmp

array([3.259112 , 4.9346   , 4.949    , 4.8962   , 4.5199   , 0.1173   ,
       4.4035   , 4.9898   , 1.9302682, 0.1127   ], dtype=float32)

In [15]:
Loss_tmp

9.944092750549316

In [18]:
Prediction.preds_for_one_model(MVGP_models, MVGP_likelihoods, torch.tensor(estimated_params_tmp).cuda().unsqueeze(0))

tensor([-0.0063,  1.3786,  2.2996,  4.0408,  1.5729,  2.0838, -0.6114,  0.1411,
         0.6820, -0.1512,  0.2804,  0.0749], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [19]:
input_point

tensor([  3.5505, -16.6149,  -4.1116,  -0.1477,   3.0083,  -4.4681,  -0.6690,
         -0.7311,  -0.7836,   0.3199,   0.5549,  -0.2954])

In [22]:
row_idx = 0

input_point = test_y_pca_peak[row_idx,:]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_GPU(input_point, train_x, train_y_pca_peak, k=100)

bounds = bound.get_bounds(local_train_x)


In [23]:

estimated_params_tmp, Loss_tmp = Estimation.multi_start_estimation(
        MVGP_models, MVGP_likelihoods, row_idx, test_y_pca_peak, bounds,
        Estimation.estimate_params_for_one_model_Adam, num_starts=16, num_iterations=2000, lr=0.01,
        patience=15, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
    )

In [24]:
estimated_params_tmp

array([4.1117377 , 4.018862  , 1.3818005 , 4.8903985 , 2.8841236 ,
       0.56604326, 3.0112777 , 3.5116904 , 4.7275767 , 1.8389444 ],
      dtype=float32)

In [25]:
Loss_tmp

0.02026066929101944

In [26]:
test_x[row_idx,:]

tensor([3.6700, 4.8642, 1.4330, 4.7406, 2.8699, 0.6171, 3.0394, 3.4495, 4.7810,
        1.8129])

In [ ]:
mcmc_result_Uniform = Estimation.run_mcmc_Uniform(
        Prediction.preds_distribution, MVGP_models, MVGP_likelihoods, 
        row_idx, test_y, bounds, 
        num_sampling=1200, warmup_step=300, num_chains=1, device=Device
    )

# Check result

In [ ]:
MVGP_pca_result = pd.read_csv('RealCase/Result/MVGP_pca_result.csv')
MVGP_pca_result['estimated_params_list'] = MVGP_pca_result['estimated_params'].apply(ast.literal_eval)

MVGP_pca_result = np.array(MVGP_pca_result['estimated_params_list'].tolist())

In [ ]:
realcase_paras_pca = torch.tensor(MVGP_pca_result, dtype=torch.float32)

In [ ]:
preds_tmp = Prediction.preds_for_one_model(
        MVGP_models, MVGP_likelihoods, realcase_paras_pca.to(Device)
        ).cpu().detach().numpy()

# End